In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical
from torch.utils.data.sampler import BatchSampler, SubsetRandomSampler

from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

from comet_ml import Experiment

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
num_envs = 8
max_frames       = 10000000
lr               = 2.5e-4
num_steps        = 128
mini_batch_size  = 4
ppo_epochs       = 3
save_every_update = 10
gamma = 0.99
tau = 0.95
critic_weight = 1.0
entropy_weight = 0.01
clip_param = 0.1
num_stack = 4

save_file = "./models/ppo_pong_0.state"
resume = False
resume_file = "./models/ppo_pong_0.state"

In [4]:
experiment = Experiment(api_key="dZm2UV8sODS5eDYysEf8TzKNu", 
                        project_name="PPO_PONG")
experiment.log_multiple_params({"Num_Actors": num_envs,
                               "Max_Frames": max_frames,
                               "LR": lr,
                               "Horizon": num_steps,
                               "Num_Mini_Batches": mini_batch_size,
                               "Epochs": ppo_epochs,
                               "Gamma": gamma,
                               "Tau": tau,
                               "VF Coeff": critic_weight,
                               "Entropy Coef": entropy_weight,
                               "Clip Para": clip_param,
                               "Num Stack": num_stack})

COMET WARNING: We detected that you are running inside a Ipython/Jupyter notebook environment but we cannot save your notebook source code. Please be sure to have installed comet_ml as a notebook server extension by running:
jupyter comet_ml enable
COMET INFO: old comet version (1.0.9) detected. current: 1.0.20 please update your comet lib with command: `pip install --no-cache-dir --upgrade comet_ml`
COMET WARNING: Comet.ml support for Ipython Notebook is limited at the moment, automatic monitoring and stdout capturing is deactivated
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/syrios/ppo-pong/321206f0128a42118c56773493ac1eca



In [5]:
from common.multiprocessing_env import SubprocVecEnv

env_name = "PongNoFrameskip-v4"

def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk

envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)

env = gym.make(env_name)

In [6]:
def init(module, weight_init, bias_init, gain=1):
    weight_init(module.weight.data, gain=gain)
    bias_init(module.bias.data)
    return module

class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(ActorCritic, self).__init__()
        
        init_ = lambda m: init(m,
                      nn.init.orthogonal_,
                      lambda x: nn.init.constant_(x, 0),
                      nn.init.calculate_gain('relu'))

        self.main = nn.Sequential(
            init_(nn.Conv2d(num_inputs, 32, 8, stride=4)),
            nn.ReLU(),
            init_(nn.Conv2d(32, 64, 4, stride=2)),
            nn.ReLU(),
            init_(nn.Conv2d(64, 32, 3, stride=1)),
            nn.ReLU(),
            Flatten(),
            init_(nn.Linear(32 * 7 * 7, 512)),
            nn.ReLU())
        
        init_ = lambda m: init(m,
                               nn.init.orthogonal_,
                               lambda x: nn.init.constant_(x, 0))
        
        self.critic_linear = init_(nn.Linear(512, 1))
        self.actor_linear   = init_(nn.Linear(512, num_outputs))
                         
    def forward(self, x):
        features = self.main(x / 255.0)
        value = self.critic_linear(features)
        softs = F.softmax(self.actor_linear(features), dim=1)
        dist = Categorical(softs)
        return dist, value

In [7]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    current_obs = torch.zeros(num_envs, *obs_shape).to(device)
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        update_current_obs(np.expand_dims(state, axis=0))
        dist, _ = model(current_obs)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [8]:
def compute_gae(next_value, rewards, masks, values, gamma=gamma, tau=tau):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [9]:
obs_shape = (3 * num_stack, 84, 84)

def update_current_obs(obs):
    shape_dim0 = 3
    obs = obs.transpose(0, 3, 1, 2)
    obs = torch.from_numpy(obs).float()
    obs = nn.functional.interpolate(obs, (84,84))
    if num_stack > 1:
        current_obs[:, :-shape_dim0] = current_obs[:, shape_dim0:]
        current_obs[:, -shape_dim0:] = obs

num_outputs = envs.action_space.n

model = ActorCritic(3 * num_stack, num_outputs).to(device)
if resume:
    model.load_state_dict(torch.load(resume_file))
optimizer = optim.Adam(model.parameters(), lr=lr)

In [10]:
frame_idx    = 0
test_rewards = []

# Important

With GAE: GAE advantage just used for actor loss. Discounted returns used for critic.

In [ ]:
## grab random values for each batch
def ppo_iter(mini_batch_size, states, actions, log_probs, returns, advantage, values):
    batch_size = states.size(0)
    sampler = BatchSampler(SubsetRandomSampler(range(batch_size)), 
                           batch_size // mini_batch_size, 
                           drop_last=False)
    for rand_ids in sampler:
        yield states[rand_ids, :], actions[rand_ids, :], log_probs[rand_ids, :], returns[rand_ids, :], advantage[rand_ids, :], values[rand_ids, :]
        
        
def ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, values, clip_param=clip_param):
    for _ in range(ppo_epochs):
        for s, act, lp, r, adv, v in ppo_iter(mini_batch_size, states, actions, log_probs,
                                                                       returns, advantages, values):
            
            
            new_dist, new_value = model(s)
            new_entropy = new_dist.entropy().mean()
            new_log_prob = new_dist.log_prob(act)
            ratio = torch.exp(new_log_prob - lp)
            surr1 = ratio * adv
            surr2 = torch.clamp(ratio, 1 - clip_param, 1 + clip_param) * adv
            actor_loss = - torch.min(surr1, surr2).mean()
            critic_loss = F.mse_loss(r,v)


            optimizer.zero_grad()
            loss = (actor_loss + critic_weight * critic_loss - entropy_weight * new_entropy)
            loss.backward(retain_graph=True)
            nn.utils.clip_grad_norm_(model.parameters(), 0.5)
            optimizer.step()

In [ ]:
state = envs.reset()
update_iter = 0

while frame_idx < max_frames:
    log_probs = []
    values = []
    rewards = []
    masks = []
    states = torch.zeros(num_steps, num_envs, *obs_shape).to(device)
    actions = []
    current_obs = torch.zeros(num_envs, *obs_shape).to(device)
    
    for step_index in range(num_steps):
        # state is 16 x 4 because 16 envs
        update_current_obs(state)
        states[step_index].copy_(current_obs)
        # dist and value each have 16 for all envs
        dist, value = model(current_obs)
        
        # have 16 actions
        action = dist.sample()
        actions.append(action)
        next_state, reward, done, _ = envs.step(action.cpu().numpy())
        
        log_prob = dist.log_prob(action)

        log_probs.append(log_prob)
        values.append(value)
        # there are 16 rewards. Need to make it 16x1. Same for masks
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
                
        state = next_state
        frame_idx += 1

    update_current_obs(next_state)
    _, next_value = model(current_obs)
    returns = compute_gae(next_value, rewards, masks, values)
    
    log_probs = torch.cat(log_probs).unsqueeze(1).to(device)
    returns = torch.cat(returns).detach().to(device)
    values = torch.cat(values).to(device)
    advantages = returns - values
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-5)
    actions   = torch.cat(actions).unsqueeze(1).to(device)
    states = states.view(-1, 3*num_stack, 84, 84)
                        
    ppo_update(ppo_epochs, mini_batch_size, states, actions, log_probs, returns, advantages, values)
    
    test_reward = test_env(False)
    experiment.log_metric("reward", test_reward,
                          step=update_iter)
    
    if update_iter % save_every_update == 0:
            clear_output(True)
            print("Update Iteration: {}".format(update_iter))
            print("Frames Processed: {}".format(frame_idx))
            print("Last Test Reward: {}".format(test_reward))
            print("Saving Model...")
            torch.save(model.state_dict(), save_file)
            
    update_iter += 1

Update Iteration: 5030
Frames Processed: 643968
Last Test Reward: -21.0
Saving Model...


In [ ]:
test_env(True)